In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Flatten, Dense, Input, Conv2D, concatenate, BatchNormalization, MaxPooling2D, UpSampling2D, Concatenate, Dropout, Cropping2D, ZeroPadding2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import jaccard_score
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.metrics import MeanIoU

In [5]:
TRAIN_IMAGE_DIR = "/kaggle/input/diabetic-retinopathy-dataset/Daataset_DR/DB0/Images"
TRAIN_MASK_DIR = "/kaggle/input/diabetic-retinopathy-dataset/Daataset_DR/DB0/GroundTruth"
TEST_IMAGE_DIR = "/kaggle/input/diabetic-retinopathy-dataset/Daataset_DR/DB1/Images"
TEST_MASK_DIR = "/kaggle/input/diabetic-retinopathy-dataset/Daataset_DR/DB1/GroundTruth"
TRAIN_MASKS = "/kaggle/working/CombinedMasksTask2"
TEST_MASKS = "/kaggle/working/TestCombinedMasksTask2"

In [6]:
def create_and_save_combined_masks(output_mask_dir, mask_classes, train_image_dir, train_mask_dir):
    os.makedirs(output_mask_dir, exist_ok=True)

    # Get list of image filenames in the training image directory
    image_filenames = os.listdir(train_image_dir)

    # Process each imagef
    for image_filename in tqdm(image_filenames):
        combined_mask = np.zeros(cv2.imread(os.path.join(train_mask_dir, mask_classes[0], image_filename), cv2.IMREAD_GRAYSCALE).shape, dtype=np.uint8)
        for subfolder in mask_classes:
            mask_path = os.path.join(train_mask_dir, subfolder, image_filename)
            if os.path.exists(mask_path):
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                combined_mask = cv2.bitwise_or(combined_mask, mask)

        output_path = os.path.join(output_mask_dir, image_filename)
        cv2.imwrite(output_path, combined_mask)

In [7]:
mask_subfolders = ['H', 'MA']
create_and_save_combined_masks(TRAIN_MASKS, mask_subfolders, TRAIN_IMAGE_DIR, TRAIN_MASK_DIR)
create_and_save_combined_masks(TEST_MASKS, mask_subfolders, TEST_IMAGE_DIR, TEST_MASK_DIR)

100%|██████████| 89/89 [00:02<00:00, 30.45it/s]


In [8]:
def load_data(image_dir, mask_dir):
    image_filenames = sorted(os.listdir(image_dir))
    images = []
    masks = []
    
    for filename in tqdm(image_filenames, desc="Loading data"):
        # Load and resize image
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path)
        images.append(image)  # Normalize image to [0, 1]
        
        mask_path = os.path.join(mask_dir, filename)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = (mask > 0).astype(np.uint8)  # Binarize mask (0 or 1)
        masks.append(mask)
    
    return np.array(images), np.array(masks)

In [9]:
images, masks = load_data(TRAIN_IMAGE_DIR, TRAIN_MASKS)

X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.1, random_state=42)

Loading data: 100%|██████████| 130/130 [00:07<00:00, 16.42it/s]


In [10]:
def data_patches(image_data):
    # Define patch size
    patch_size = 16

    # Get the dimensions of the image
    num_rows, num_cols, num_bands = image_data.shape

    # Initialize an empty list to store patches
    patches = []

    # Loop over the image in patches of 16x16
    for row in range(0, num_rows, patch_size):
        for col in range(0, num_cols, patch_size):
            # Extract the patch
            patch = image_data[row:row+patch_size, col:col+patch_size]

            # Check if the patch has the right shape (i.e., it's not on the edge)
            if patch.shape[:2] == (patch_size, patch_size):
                patches.append(patch)

    # Convert list of patches to numpy array
    patches_array = np.array(patches)
    return patches_array

In [11]:
train_patches = []

for a in X_train:
    patches = data_patches(a)
    train_patches.append(patches)

train_patches = np.concatenate(train_patches, axis=0)
train_patches.shape

(783432, 16, 16, 3)

In [12]:
validation_patches = []
for a in X_val:
    patches = data_patches(a)
    validation_patches.append(patches)

validation_patches = np.concatenate(validation_patches, axis=0)
validation_patches.shape

(87048, 16, 16, 3)

In [13]:
def label_patches2(image):
    patch_size = 16
    labeled_patches = []  # List to hold the label of each patch

    # Loop over the image in patches of 8x8
    for row in range(0, image.shape[0], patch_size):
        for col in range(0, image.shape[1], patch_size):
            # Extract the patch
            patch = image[row:row + patch_size, col:col + patch_size]

            # Check if the patch has the correct shape (i.e., it's not on the edge)
            if patch.shape[:2] == (patch_size, patch_size):
                # Check if there is at least one pixel with the value 1
                if 1 in patch:
                    labeled_patches.append(1)
                else:
                    labeled_patches.append(0)

    return np.array(labeled_patches)

In [14]:
train_labels = []

for a in y_train:
    labels = label_patches2(a)
    train_labels.extend(labels)

train_labels = np.array(train_labels)
train_labels.shape

(783432,)

In [15]:
validation_labels = []

for a in y_val:
    labels = label_patches2(a)
    validation_labels.extend(labels)

validation_labels = np.array(validation_labels)
validation_labels.shape

(87048,)

In [16]:
def build_model():
    model = Sequential()
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', input_shape=(16, 16, 3)))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(1024, (3, 3), activation='relu', padding='same'))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Model summary to check structure
    return model
model = build_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Setup the ModelCheckpoint callback to save the best model based on validation accuracy
checkpoint = ModelCheckpoint(
    'best_model.keras',  # Path where the model is saved
    monitor='val_accuracy',  # Monitor validation accuracy
    save_best_only=True,  # Only save the model if 'val_accuracy' has improved
    mode='max',  # 'max' because we want to maximize validation accuracy
    verbose=1  # Optional: provides detailed logging about the saved models
)

# Fit the model using the previously defined training and validation data
history2 = model.fit(
    train_patches, 
    train_labels, 
    validation_data=(validation_patches, validation_labels),
    epochs=50,
    batch_size=2048,
    callbacks=[checkpoint]  # Include the checkpoint in the callbacks
)

Epoch 1/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 0.9820 - loss: 0.0767
Epoch 1: val_accuracy improved from -inf to 0.99045, saving model to best_model.keras
383/383 ━━━━━━━━━━━━━━━━━━━━ 188s 383ms/step - accuracy: 0.9820 - loss: 0.0766 - val_accuracy: 0.9905 - val_loss: 0.0564
Epoch 2/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - accuracy: 0.9952 - loss: 0.0352
Epoch 2: val_accuracy did not improve from 0.99045
383/383 ━━━━━━━━━━━━━━━━━━━━ 102s 265ms/step - accuracy: 0.9952 - loss: 0.0352 - val_accuracy: 0.9905 - val_loss: 0.0620
Epoch 3/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - accuracy: 0.9952 - loss: 0.0339
Epoch 3: val_accuracy did not improve from 0.99045
383/383 ━━━━━━━━━━━━━━━━━━━━ 102s 265ms/step - accuracy: 0.9952 - loss: 0.0339 - val_accuracy: 0.9905 - val_loss: 0.0523
Epoch 4/50
383/383 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - accuracy: 0.9954 - loss: 0.0288
Epoch 4: val_accuracy improved from 0.99045 to 0.99055, saving model to best_model.keras
383/383 

In [ ]:
IMAGE_DIR = "/kaggle/input/diabetic-retinopathy-dataset/Daataset DR/DB1/Images"
MASK_DIR = "/kaggle/working/TestCombinedMasksTask2"
def load_data(image_dir, mask_dir):
    image_filenames = sorted(os.listdir(image_dir))
    images = []
    masks = []
    
    for filename in tqdm(image_filenames, desc="Loading data"):
        # Load and resize image
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path)
        images.append(image)
        
        mask_path = os.path.join(mask_dir, filename)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = (mask > 0).astype(np.uint8)  # Binarize mask (0 or 1)
        masks.append(mask)
    
    return np.array(images), np.array(masks)

X_test, y_test = load_data(IMAGE_DIR, MASK_DIR)

In [ ]:
test_patches = []

for a in X_test:
    patches = data_patches(a)
    test_patches.append(patches)

test_patches = np.concatenate(test_patches, axis=0)
test_patches.shape

In [ ]:
test_labels = []

for a in y_test:
    labels = label_patches2(a)
    test_labels.extend(labels)

test_labels = np.array(test_labels)
test_labels.shape

In [ ]:
model.evaluate(test_patches,test_labels)

In [ ]:
predictions = model.predict(test_patches)

In [ ]:
predict = np.argmax(predictions, axis=1)

In [ ]:
pred = []
for x in predict:
  pred.append(np.full((16, 16), x))
pred = np.array(pred)

In [ ]:
pred2 = pred.reshape(89, 6696, 16,16)

In [ ]:
predicted_masks = []
for a in pred2:
    patches_per_row = 93
    patches_per_col = len(a) // patches_per_row
    image = np.concatenate([np.concatenate(a[i:i+patches_per_row], axis=1) for i in range(0, len(a), patches_per_row)], axis=0)
    predicted_masks.append(image)

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'predicted_masks' is your list or array of images
num_images = 89
rows = 18
cols = 5

fig, axes = plt.subplots(rows, cols, figsize=(15, 54))  # Adjust figsize to your screen/display size
axes = axes.flatten()  # Flatten the 2D array of axes to simplify the looping

# Loop over all of the positions in the grid
for i in range(rows * cols):
    if i < num_images:
        # Display image
        axes[i].imshow(predicted_masks[i], cmap='gray')  # Assuming masks are grayscale
        axes[i].axis('off')  # Turn off axis numbering and ticks
    else:
        axes[i].axis('off')  # Make sure empty plots also have no axes

plt.tight_layout()  # Optional, improves spacing between plots
plt.show()

In [ ]:
import matplotlib.image as mpimg

test_image_dir = "/kaggle/input/diabetic-retinopathy-dataset/Daataset DR/DB1/Images"
ground_truth_dir = "/kaggle/working/TestCombinedMasksTask2"
image_files = sorted(os.listdir(test_image_dir))
ground_truth_files = sorted(os.listdir(ground_truth_dir))

assert len(image_files) == len(ground_truth_files) == 89, "Mismatch in number of files"

num_images = len(image_files)
fig, axes = plt.subplots(num_images, 3, figsize=(15, 5 * num_images))

for i, image_file in enumerate(image_files):
    # Read the image and the ground truth mask
    img_path = os.path.join(test_image_dir, image_file)
    gt_path = os.path.join(ground_truth_dir, ground_truth_files[i])

    image = mpimg.imread(img_path)
    ground_truth = mpimg.imread(gt_path)
    predicted_mask = predicted_masks[i]

    # Plotting
    axes[i, 0].imshow(image)
    axes[i, 0].set_title('Original Image')
    axes[i, 0].axis('off')

    axes[i, 1].imshow(ground_truth)
    axes[i, 1].set_title('Ground Truth Mask')
    axes[i, 1].axis('off')

    axes[i, 2].imshow(predicted_mask)
    axes[i, 2].set_title('Predicted Mask')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
padded_masks = np.array([np.pad(mask, pad_width=((0, 0), (0, 12)), mode='constant', constant_values=0) for mask in predicted_masks])

In [ ]:
def iou_score(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred).sum()
    union = np.logical_or(y_true, y_pred).sum()
    if union == 0:
        return 1.0  # To handle cases with no ground truth or predicted objects
    else:
        return intersection / union

In [ ]:
def iou_score(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred).sum()
    union = np.logical_or(y_true, y_pred).sum()
    if union == 0:
        return 1.0  # To handle cases with no ground truth or predicted objects
    else:
        return intersection / union

# ***Task2 END***